# This is Example Code
I'm going to try and write a bit of code that will take in position and angle arguments to one or more sources in a cubic room and return the position of the sources.

In [ ]:
import numpy as np                         # np is the alias we can now use to access any tool in numpy
import math
from matplotlib import pyplot as plt       # here, we only need the pyplot module from the library (with alias plt)
import pandas as pd
import scipy
from scipy import special                  # import the special functions, such as "gamma functions"
import scipy.optimize as opt               # this is needed for chi-squared minimization
from scipy.optimize import curve_fit       # this is the curve-fitting function
from itertools import product
from matplotlib import pyplot as plt

In [ ]:
room = np.zeros(shape=(1000,1000,1000)) #this creates a virtual room that is filled with zeros (representing zero detected sources at each point).

In [ ]:
def create_lines(position, angle, room_state):
  """
  This function will return the room matrix with 1s in all of the possible locations for radiation sources
    (i.e. all of the spaces where a radiation source could be to give the angle at the position)

  Inputs:
    position - a vector containing the x, y, and z position of the robot
    angle - a vector containing the mu and gamma for the angle from the robot to the source (with straight ahead being <0,pi/2> and going counter-clockwise)
    room_state - the 3d room matrix, which is updated and returned.
  """
  mu = angle[0]
  gamma = angle[1]

  step_x = np.sqrt(1 - mu ** 2) * np.cos(gamma) #defines the x component of the unit vector
  step_y = np.sqrt(1 - mu ** 2) * np.sin(gamma) #defines the y component of the unit vector
  step_z = mu                                   #defines the z component of the unit vector

  x_coordinate = position[0]
  y_coordinate = position[1]
  z_coordinate = position[2]

  in_bounds = True #
  while in_bounds:
    x_coordinate += step_x
    y_coordinate += step_y
    z_coordinate += step_z

    pl[math.ceil(x_coordinate)][math.ceil(y_coordinate)][math.ceil(z_coordinate)] += 1

    if abs(x_coordinate - 500) > 498: #if the latest position is too close to the walls, call off the whole thing (this is bad code to avoid index errors)
      in_bounds = False
    if abs(y_coordinate - 500) > 498:
      in_bounds = False
    if abs(z_coordinate - 500) > 498:
      in_bounds = False

  return room_state

def get_angle(robot_location, source_location):
  dx = source_location[0] - robot_location[0]
  dy = source_location[1] - robot_location[1]
  dz = source_location[2] - robot_location[2]
  r = np.sqrt(dx ** 2 + dy ** 2 + dz ** 2)

  mu = dz / r
  if dx != 0:
    theta = np.arctan(dy/dx)
  else:
    theta = np.pi / 2

  return [mu, theta]

In [ ]:
source1 = [500,550,600]

robot_location1 = [500,500,5]
robot_location2 = [5,900,10]
robot_location3 = [500,10,10]

room = create_lines(robot_location1, get_angle(robot_location1, source1), room)
room = create_lines(robot_location2, get_angle(robot_location2, source1), room)
room = create_lines(robot_location3, get_angle(robot_location3, source1), room)

In [ ]:
rad_locations = np.nonzero(room)
space = np.array(rad_locations)
intensities = np.zeros(len(space[0,:]))
for i in range(0, len(intensities),1):
  intensities[i] = room[space[0,i], space[1,i], space[2,i]]

In [1]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(projection="3d")
ax.scatter(source1[0], source1[1], source1[2], s = 100, c = 'Red')

ax.scatter(space[0,:], space[1,:], space[2,:], s = intensities)
ax.scatter(robot_location1[0], robot_location1[1], robot_location1[2], s = 100, c = 'Black')
ax.scatter(robot_location2[0], robot_location2[1], robot_location2[2], s = 100, c = 'Green')
ax.scatter(robot_location3[0], robot_location3[1], robot_location3[2], s = 100, c = 'Yellow')

NameError: name 'plt' is not defined

In [ ]:
blurred_image = scipy.ndimage.gaussian_filter(room, sigma = 0)

In [ ]:
N = len(blurred_image)
max_val = 0
max_location = [-1, -1, -1]
for i in range(0, N, 1):
  for j in range(0, N, 1):
    for k in range(0, N, 1):
      if blurred_image[i,j,k] > max_val:
        max_val = blurred_image[i,j,k]
        max_location = [i,j,k]

print(max_location)

[500, 551, 601]
